In [1]:
from sklearn.model_selection import KFold, train_test_split
import torch.distributions.distribution as D
import data.medain_filtering_class as mf
from torch.utils.data import DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
from RBM import RBM
import numpy as np
import datetime
import torch

print(datetime.datetime.now(), "model.py code start")

BATCH_SIZE = 10
EPOCH = 100
LEARNING_RATE = 0.2
ANNEALING_RATE = 0.999
VISIBLE_UNITS = [180, 200, 250]
HIDDEN_UNITS = [80, 100, 120]
K_FOLD = 1

2022-05-05 12:14:57.454245 model.py code start


In [2]:
print("[MODL] Model main code is starting....")

print("[INFO] Read train data, cross-vaildation data and test data from median filtering code")
dataset_db1, dataset_db2, dataset_db3 = mf.ecg_filtering(True)

train_dataset = list(mf.list_to_list(dataset_db1)) * 4
cross_dataset = list(mf.list_to_list(dataset_db2)) * 4
test_dataset = list(mf.list_to_list(dataset_db3))  * 4

X_train, X_test, y_train, y_test = train_test_split(
    (train_dataset + cross_dataset), 
    (test_dataset + cross_dataset),
    test_size=0.33,
    shuffle=True
)

[MODL] Model main code is starting....
[INFO] Read train data, cross-vaildation data and test data from median filtering code
[INFO] Read file and indexing start...
[INFO]	final_db1 direcotry found.
......	...................i	Current_Index	From_Index
[IWIP]	final_db1 reading... 0 0 200
[IWIP]	final_db1 reading... 1 200 400
[IWIP]	final_db1 reading... 2 400 600
[IWIP]	final_db1 reading... 3 600 800
[IWIP]	final_db1 reading... 4 800 1000
[IWIP]	final_db1 reading... 5 1000 1200
[IWIP]	final_db1 reading... 6 1200 1400
[IWIP]	final_db1 reading... 7 1400 1600
[IWIP]	final_db1 reading... 8 1600 1800
[IWIP]	final_db1 reading... 9 1800 2000
[IWIP]	final_db1 reading... 10 2000 2200
[IWIP]	final_db1 reading... 11 2200 2400
[IWIP]	final_db1 reading... 12 2400 2600
[IWIP]	final_db1 reading... 13 2600 2800
[IWIP]	final_db1 reading... 14 2800 3000
[IWIP]	final_db1 reading... 15 3000 3200
[IWIP]	final_db1 reading... 16 3200 3400
[IWIP]	final_db1 reading... 17 3400 3600
[IWIP]	final_db1 reading... 18 

/Users/bahk_insung/miniforge3/lib/python3.9/site-packages/scipy/signal/_signaltools.py:1531: UserWarning: kernel_size exceeds volume extent: the volume will be zero-padded.
  warnings.warn('kernel_size exceeds volume extent: the volume will be '


In [3]:
train_dataloader = DataLoader(X_train + y_train,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

test_dataloader = DataLoader(X_test + y_test,
                             batch_size=BATCH_SIZE,
                             shuffle=True)

print("X_train length : ", len(X_train))
print("X_test  length : ", len(X_test))
print("y_train length : ", len(y_train))
print("y_test  length : ", len(y_test))

train_data = torch.FloatTensor(X_train)
test_data = torch.FloatTensor(X_test)

X_train length :  9648
X_test  length :  4752
y_train length :  9648
y_test  length :  4752


In [4]:
print("[INFO] Model object added")

rbm_first = RBM(n_vis=VISIBLE_UNITS[0], n_hid=HIDDEN_UNITS[0], k=K_FOLD, batch=BATCH_SIZE)
rbm_second = RBM(n_vis=VISIBLE_UNITS[1], n_hid=HIDDEN_UNITS[1], k=K_FOLD, batch=BATCH_SIZE)
rbm_third = RBM(n_vis=VISIBLE_UNITS[2], n_hid=HIDDEN_UNITS[2], k=K_FOLD, batch=BATCH_SIZE)

first_train_op = optim.SGD(rbm_first.parameters(), 0.1)
second_train_op = optim.SGD(rbm_second.parameters(), 0.1)
third_train_op = optim.SGD(rbm_third.parameters(), 0.1)

gb_first_train_op = optim.SGD(rbm_first.parameters(), 0.1)
gb_second_train_op = optim.SGD(rbm_second.parameters(), 0.1)
gb_third_train_op = optim.SGD(rbm_third.parameters(), 0.1)

output_from_first = list()
output_from_second = list()
output_from_third = list()

[INFO] Model object added


In [5]:
'''Train Part'''

loss_ = []
for epoch in range(EPOCH):
    '''First bbrbm'''
    for _, (data) in enumerate(train_dataloader):
        try:
            # tnesor float
            data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        except RuntimeError:
            continue

        sample_data = torch.bernoulli(data)
        sample_data = torch.flatten(sample_data.clone())

        # tensor binary
        vog_first, v1, mt = rbm_first(sample_data)
        
        loss_first = rbm_first.free_energy(vog_first) - rbm_first.free_energy(v1)
        loss_.append(loss_first.data)
        
        first_train_op.zero_grad()
        loss_first.backward()
        first_train_op.step()
    
    output_from_first.append(v1.tolist())
    print("1ST BBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}".format(epoch, np.mean(loss_), mt))

output_from_first = torch.tensor(output_from_first)
for epoch in range(EPOCH):
    '''Secnd bbrbm'''
    for _, (data) in enumerate(output_from_first):
        try:
            data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        except RuntimeError:
            continue

        sample_data = torch.bernoulli(data)
        sample_data = torch.flatten(sample_data.clone())

        vog_second, v2, mt = rbm_second(sample_data)
        
        loss_second = rbm_second.free_energy(vog_second) - rbm_second.free_energy(v2)
        loss_.append(loss_second.data)
        
        second_train_op.zero_grad()
        loss_second.backward()
        second_train_op.step()

    output_from_second.append(v2.tolist())
    print("2ST BBrbm_first Training loss for {0} epoch {1}\tEstimate time : ".format(epoch, np.mean(loss_), mt))

output_from_second = torch.tensor(output_from_second)
for epoch in range(EPOCH):
    '''Third bbrbm'''
    for _, (data) in enumerate(output_from_second):
        try:
            data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        except RuntimeError:
            continue

        sample_data = torch.bernoulli(data)
        sample_data = torch.flatten(sample_data.clone())

        vog_third, v3, mt = rbm_third(sample_data)
        
        loss_third = rbm_third.free_energy(vog_third) - rbm_third.free_energy(v3)
        loss_.append(loss_third.data)
        
        third_train_op.zero_grad()
        loss_third.backward()
        third_train_op.step()

    output_from_third.append(v3.tolist())
    print("3ST BBrbm_first Training loss for {0} epoch {1}\tEstimate time : ".format(epoch, np.mean(loss_), mt))

/var/folders/_8/q_cwh5hn0s1dxsrzq2d040p80000gn/T/ipykernel_51227/3670610706.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
/Users/bahk_insung/miniforge3/lib/python3.9/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


1ST BBrbm_first Training loss for 0 epoch -0.16360503435134888	Estimate time : 0:00:00.000139
1ST BBrbm_first Training loss for 1 epoch -0.02275623008608818	Estimate time : 0:00:00.000136
1ST BBrbm_first Training loss for 2 epoch -0.011981554329395294	Estimate time : 0:00:00.000136
1ST BBrbm_first Training loss for 3 epoch -0.07441113144159317	Estimate time : 0:00:00.000137
1ST BBrbm_first Training loss for 4 epoch -0.021670663729310036	Estimate time : 0:00:00.000142
1ST BBrbm_first Training loss for 5 epoch 0.011204813607037067	Estimate time : 0:00:00.000139
1ST BBrbm_first Training loss for 6 epoch -0.010123432613909245	Estimate time : 0:00:00.000138
1ST BBrbm_first Training loss for 7 epoch -0.039840638637542725	Estimate time : 0:00:00.000139
1ST BBrbm_first Training loss for 8 epoch -0.08644601702690125	Estimate time : 0:00:00.000137
1ST BBrbm_first Training loss for 9 epoch -0.03612577170133591	Estimate time : 0:00:00.000136
1ST BBrbm_first Training loss for 10 epoch -0.0608907751

/var/folders/_8/q_cwh5hn0s1dxsrzq2d040p80000gn/T/ipykernel_51227/3670610706.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)


2ST BBrbm_first Training loss for 0 epoch -0.005007104016840458	Estimate time : 
2ST BBrbm_first Training loss for 1 epoch -0.0051079704426229	Estimate time : 
2ST BBrbm_first Training loss for 2 epoch -0.005118563771247864	Estimate time : 
2ST BBrbm_first Training loss for 3 epoch -0.0052263145335018635	Estimate time : 
2ST BBrbm_first Training loss for 4 epoch -0.005185925401747227	Estimate time : 
2ST BBrbm_first Training loss for 5 epoch -0.005166439339518547	Estimate time : 
2ST BBrbm_first Training loss for 6 epoch -0.005153132602572441	Estimate time : 
2ST BBrbm_first Training loss for 7 epoch -0.005626448430120945	Estimate time : 
2ST BBrbm_first Training loss for 8 epoch -0.005395208019763231	Estimate time : 
2ST BBrbm_first Training loss for 9 epoch -0.005125011783093214	Estimate time : 
2ST BBrbm_first Training loss for 10 epoch -0.005656478926539421	Estimate time : 
2ST BBrbm_first Training loss for 11 epoch -0.005396674852818251	Estimate time : 
2ST BBrbm_first Training lo

/var/folders/_8/q_cwh5hn0s1dxsrzq2d040p80000gn/T/ipykernel_51227/3670610706.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)


3ST BBrbm_first Training loss for 0 epoch -0.01177135668694973	Estimate time : 
3ST BBrbm_first Training loss for 1 epoch -0.011738482862710953	Estimate time : 
3ST BBrbm_first Training loss for 2 epoch -0.012042942456901073	Estimate time : 
3ST BBrbm_first Training loss for 3 epoch -0.011990212835371494	Estimate time : 
3ST BBrbm_first Training loss for 4 epoch -0.011809495277702808	Estimate time : 
3ST BBrbm_first Training loss for 5 epoch -0.011829285882413387	Estimate time : 
3ST BBrbm_first Training loss for 6 epoch -0.011910767294466496	Estimate time : 
3ST BBrbm_first Training loss for 7 epoch -0.011820242740213871	Estimate time : 
3ST BBrbm_first Training loss for 8 epoch -0.011636699549853802	Estimate time : 
3ST BBrbm_first Training loss for 9 epoch -0.011688296683132648	Estimate time : 
3ST BBrbm_first Training loss for 10 epoch -0.011909259483218193	Estimate time : 
3ST BBrbm_first Training loss for 11 epoch -0.011699515394866467	Estimate time : 
3ST BBrbm_first Training lo

In [6]:
    
print("BBRBM is done.")
print("GBRBM is start")

output_from_first = list()
output_from_second = list()
output_from_third = torch.tensor(output_from_third)

rbm_first = RBM(n_vis=VISIBLE_UNITS[0], n_hid=HIDDEN_UNITS[0], k=K_FOLD, batch=BATCH_SIZE)
rbm_second = RBM(n_vis=VISIBLE_UNITS[1], n_hid=HIDDEN_UNITS[1], k=K_FOLD, batch=BATCH_SIZE)
rbm_third = RBM(n_vis=VISIBLE_UNITS[2], n_hid=HIDDEN_UNITS[2], k=K_FOLD, batch=BATCH_SIZE)

# print(output_from_third.size(), output_from_third.dim(), "\n", output_from_third)
gaussian_std = torch.arange(1, 0, -0.1)

for epoch in range(EPOCH):
    '''First gbrbm'''
    for _, (data) in enumerate(output_from_third):
        try:
            data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        except RuntimeError:
            continue
        
        # CHANGED to GAUSSIAN
        sample_data = torch.normal(mean=data, std=gaussian_std)
        sample_data = torch.flatten(sample_data.clone())

        gb_vog_first, gb_v1, mt = rbm_first(sample_data)
        
        gb_loss_first = rbm_first.free_energy(gb_vog_first) - rbm_first.free_energy(gb_v1)
        loss_.append(gb_loss_first.data)
        
        gb_first_train_op.zero_grad()
        gb_loss_first.backward()
        gb_first_train_op.step()

    output_from_first.append(gb_v1.tolist())
    print("1ST GBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}".format(epoch, np.mean(loss_), mt))

output_from_first = torch.tensor(output_from_first)
for epoch in range(EPOCH):
    '''Second gbrbm'''
    for _, (data) in enumerate(output_from_first):
        try:
            data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        except RuntimeError:
            continue
        
        # CHANGED to GAUSSIAN
        sample_data = torch.normal(mean=data, std=gaussian_std)
        sample_data = torch.flatten(sample_data.clone())

        gb_vog_second, gb_v2, mt = rbm_second(sample_data)
        
        gb_loss_second = rbm_second.free_energy(gb_vog_second) - rbm_second.free_energy(gb_v2)
        loss_.append(gb_loss_second.data)
        
        gb_second_train_op.zero_grad()
        gb_loss_second.backward()
        gb_second_train_op.step()

    output_from_second.append(gb_v2.tolist())
    print("2ST GBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}".format(epoch, np.mean(loss_), mt))

output_from_second = torch.tensor(output_from_second)
for epoch in range(EPOCH):
    '''Third gbrbm'''
    for _, (data) in enumerate(output_from_second):
        try:
            data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        except RuntimeError:
            continue
        
        # CHANGED to GAUSSIAN
        sample_data = torch.normal(mean=data, std=gaussian_std)
        sample_data = torch.flatten(sample_data.clone())

        gb_vog_third, gb_v3, mt = rbm_third(sample_data)
        
        gb_loss_third = rbm_third.free_energy(gb_vog_third) - rbm_second.free_energy(gb_v3)
        loss_.append(gb_loss_third.data)
        
        gb_third_train_op.zero_grad()
        gb_loss_third.backward()
        gb_third_train_op.step()

    print("3ST GBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}".format(epoch, np.mean(loss_), mt))


BBRBM is done.
GBRBM is start


/var/folders/_8/q_cwh5hn0s1dxsrzq2d040p80000gn/T/ipykernel_51227/2809123372.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)


1ST GBrbm_first Training loss for 0 epoch -0.012983662076294422	Estimate time : 0:00:00.000137
1ST GBrbm_first Training loss for 1 epoch -0.012978272512555122	Estimate time : 0:00:00.000347
1ST GBrbm_first Training loss for 2 epoch -0.012972969561815262	Estimate time : 0:00:00.000135
1ST GBrbm_first Training loss for 3 epoch -0.012967509217560291	Estimate time : 0:00:00.000134
1ST GBrbm_first Training loss for 4 epoch -0.012962261214852333	Estimate time : 0:00:00.000155
1ST GBrbm_first Training loss for 5 epoch -0.01295719388872385	Estimate time : 0:00:00.000141
1ST GBrbm_first Training loss for 6 epoch -0.012951974757015705	Estimate time : 0:00:00.000139
1ST GBrbm_first Training loss for 7 epoch -0.012946518138051033	Estimate time : 0:00:00.000149
1ST GBrbm_first Training loss for 8 epoch -0.01294123474508524	Estimate time : 0:00:00.000135
1ST GBrbm_first Training loss for 9 epoch -0.01293581910431385	Estimate time : 0:00:00.000135
1ST GBrbm_first Training loss for 10 epoch -0.0129305

/var/folders/_8/q_cwh5hn0s1dxsrzq2d040p80000gn/T/ipykernel_51227/2809123372.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)


2ST GBrbm_first Training loss for 0 epoch -0.01246892660856247	Estimate time : 0:00:00.000149
2ST GBrbm_first Training loss for 1 epoch -0.012458715587854385	Estimate time : 0:00:00.000134
2ST GBrbm_first Training loss for 2 epoch -0.012445585802197456	Estimate time : 0:00:00.000136
2ST GBrbm_first Training loss for 3 epoch -0.012433242984116077	Estimate time : 0:00:00.000135
2ST GBrbm_first Training loss for 4 epoch -0.01242332998663187	Estimate time : 0:00:00.000137
2ST GBrbm_first Training loss for 5 epoch -0.012411748059093952	Estimate time : 0:00:00.000134
2ST GBrbm_first Training loss for 6 epoch -0.012400600127875805	Estimate time : 0:00:00.000136
2ST GBrbm_first Training loss for 7 epoch -0.012388300150632858	Estimate time : 0:00:00.000139
2ST GBrbm_first Training loss for 8 epoch -0.012378147803246975	Estimate time : 0:00:00.000138
2ST GBrbm_first Training loss for 9 epoch -0.012366879731416702	Estimate time : 0:00:00.000139
2ST GBrbm_first Training loss for 10 epoch -0.012354

/var/folders/_8/q_cwh5hn0s1dxsrzq2d040p80000gn/T/ipykernel_51227/2809123372.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)


3ST GBrbm_first Training loss for 0 epoch -0.011385837569832802	Estimate time : 0:00:00.000134
3ST GBrbm_first Training loss for 1 epoch -0.011419684626162052	Estimate time : 0:00:00.000153
3ST GBrbm_first Training loss for 2 epoch -0.011454452760517597	Estimate time : 0:00:00.000419
3ST GBrbm_first Training loss for 3 epoch -0.011483965441584587	Estimate time : 0:00:00.001920
3ST GBrbm_first Training loss for 4 epoch -0.011520619504153728	Estimate time : 0:00:00.000136
3ST GBrbm_first Training loss for 5 epoch -0.0115559296682477	Estimate time : 0:00:00.000135
3ST GBrbm_first Training loss for 6 epoch -0.011591351591050625	Estimate time : 0:00:00.000134
3ST GBrbm_first Training loss for 7 epoch -0.011623821221292019	Estimate time : 0:00:00.000143
3ST GBrbm_first Training loss for 8 epoch -0.011661943048238754	Estimate time : 0:00:00.000142
3ST GBrbm_first Training loss for 9 epoch -0.011695782653987408	Estimate time : 0:00:00.000136
3ST GBrbm_first Training loss for 10 epoch -0.011731

In [7]:
nprst = gb_v3.detach().numpy()
print(nprst)

rbm_first.get_weight()

[1. 1. 1. 1. 0. 0. 0. 1. 1. 1.]


Parameter containing:
tensor([[ 0.0107, -0.0088,  0.0021,  0.0003,  0.0040, -0.0014,  0.0003,  0.0024,
         -0.0015, -0.0100]], requires_grad=True)

In [8]:
test_loss = 0
train_loss = 0
train_cnt = 0
summary_c = 0

for _, test_data in enumerate(test_dataloader):
    try:
        test_data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        train_data = torch.tensor((Variable(train_data[train_cnt:train_cnt + 10])).uniform_(0, 1), dtype=torch.float32)
    except RuntimeError:
        pass
    
    testing_data = torch.flatten(torch.bernoulli(test_data))
    training_data = torch.flatten(torch.bernoulli(train_data))
    
    vt, vt1, _ = rbm_first(testing_data)
    test_loss = rbm_first.free_energy(vt) - rbm_first.free_energy(vt1)    
    
    vs, vs1, _ = rbm_first(training_data)
    train_loss = rbm_first.free_energy(vs) - rbm_first.free_energy(vs1)

    
    test_loss += torch.mean(torch.abs(vt1[vt1 >= 0] - vt[vt1 >= 0]))
    # print(vt1[vt1 >= 0] - vt[vt1 >= 0])
    summary_c += 1

print('Test loss : ' + str(test_loss / summary_c))
print('Train - test : ' + str(train_loss - test_loss))


/var/folders/_8/q_cwh5hn0s1dxsrzq2d040p80000gn/T/ipykernel_51227/200555237.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
/var/folders/_8/q_cwh5hn0s1dxsrzq2d040p80000gn/T/ipykernel_51227/200555237.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_data = torch.tensor((Variable(train_data[train_cnt:train_cnt + 10])).uniform_(0, 1), dtype=torch.float32)


Test loss : tensor(0.0008, grad_fn=<DivBackward0>)
Train - test : tensor(-0.8000, grad_fn=<SubBackward0>)


In [9]:
'''Test code'''
rbm_first = RBM(n_vis=VISIBLE_UNITS[0], n_hid=HIDDEN_UNITS[0], k=K_FOLD, batch=BATCH_SIZE)
rbm_second = RBM(n_vis=VISIBLE_UNITS[1], n_hid=HIDDEN_UNITS[1], k=K_FOLD, batch=BATCH_SIZE)
rbm_third = RBM(n_vis=VISIBLE_UNITS[2], n_hid=HIDDEN_UNITS[2], k=K_FOLD, batch=BATCH_SIZE)

output_from_first = list()
output_from_second = list()
output_from_third = list()

test_loss = 0
epoch_cnt = 0

In [10]:
'''First BBRBM Guide Line'''

for _, data in enumerate(test_dataloader):
    try:
        test_data = torch.tensor(Variable(data.clone().detach().requires_grad_(True).view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
    except RuntimeError:
        pass
    
    data = torch.flatten(torch.bernoulli(test_data))
    
    v1, vt1, _ = rbm_first(data)
    test_loss += rbm_first.free_energy(v1) - rbm_first.free_energy(vt1)
    epoch_cnt += 1
    output_from_first.append(vt1.tolist())
print('\tBBRBM_First_layer test loss : ', str(test_loss / epoch_cnt))

'''Second BBRBM Guide Line'''

for _, data in enumerate(torch.tensor(output_from_first)):
    try:
        test_data = torch.tensor(Variable(data.clone().detach().requires_grad_(True).view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
    except RuntimeError:
        pass
    
    data = torch.flatten(torch.bernoulli(test_data))
    
    v2, vt2, _ = rbm_second(data)
    test_loss += rbm_second.free_energy(v2) - rbm_second.free_energy(vt2)
    epoch_cnt += 1
    output_from_second.append(vt2.tolist())
print('\tBBRBM_Second_layer test loss : ', str(test_loss / epoch_cnt))

'''Third BBRBM Guide Line'''

for _, data in enumerate(torch.tensor(output_from_second)):
    try:
        test_data = torch.tensor(Variable(data.clone().detach().requires_grad_(True).view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
    except RuntimeError:
        pass
    
    data = torch.flatten(torch.bernoulli(test_data))
    
    v3, vt3, _ = rbm_third(data)
    test_loss += rbm_third.free_energy(v3) - rbm_third.free_energy(vt3)
    epoch_cnt += 1
    output_from_third.append(vt3.tolist())
print('\tBBRBM_Second_layer test loss : ', str(test_loss / epoch_cnt))

	BBRBM_First_layer test loss :  tensor(0.0109, grad_fn=<DivBackward0>)
	BBRBM_Second_layer test loss :  tensor(0.0033, grad_fn=<DivBackward0>)
	BBRBM_Second_layer test loss :  tensor(0.0052, grad_fn=<DivBackward0>)


In [11]:
'''Test code'''
rbm_first = RBM(n_vis=VISIBLE_UNITS[0], n_hid=HIDDEN_UNITS[0], k=K_FOLD, batch=BATCH_SIZE)
rbm_second = RBM(n_vis=VISIBLE_UNITS[1], n_hid=HIDDEN_UNITS[1], k=K_FOLD, batch=BATCH_SIZE)
rbm_third = RBM(n_vis=VISIBLE_UNITS[2], n_hid=HIDDEN_UNITS[2], k=K_FOLD, batch=BATCH_SIZE)

output_from_first = list()
output_from_second = list()

test_loss = 0
epoch_cnt = 0

In [12]:
'''First GBRBM Guide Line'''
epoch_cnt = 0
for _, data in enumerate(torch.tensor(output_from_third)):
    try:
        test_data = torch.tensor(Variable(data.clone().detach().requires_grad_(True).view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
    except RuntimeError:
        pass
    
    sample_data = torch.flatten(torch.normal(mean=data, std=gaussian_std))

    v1, vt1, _ = rbm_first(sample_data)
    test_loss += rbm_first.free_energy(v1) - rbm_first.free_energy(vt1)
    epoch_cnt += 1
    output_from_first.append(vt1.tolist())
print('\tGBRBM_First_layer test loss : ', str(test_loss / epoch_cnt))

'''Second BBRBM Guide Line'''

for _, data in enumerate(torch.tensor(output_from_first)):
    try:
        test_data = torch.tensor(Variable(data.clone().detach().requires_grad_(True).view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
    except RuntimeError:
        pass
    
    sample_data = torch.flatten(torch.normal(mean=data, std=gaussian_std))
    
    v2, vt2, _ = rbm_second(sample_data)
    test_loss += rbm_second.free_energy(v2) - rbm_second.free_energy(vt2)
    epoch_cnt += 1
    output_from_second.append(vt2.tolist())
print('\tGBRBM_Second_layer test loss : ', str(test_loss / epoch_cnt))

'''Third BBRBM Guide Line'''

output_from_third = []
for _, data in enumerate(torch.tensor(output_from_second)):
    try:
        test_data = torch.tensor(Variable(data.clone().detach().requires_grad_(True).view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
    except RuntimeError:
        pass
    
    sample_data = torch.flatten(torch.bernoulli(test_data))
    
    v3, vt3, _ = rbm_third(sample_data)
    test_loss += rbm_third.free_energy(v3) - rbm_third.free_energy(vt3)
    epoch_cnt += 1
    output_from_third.append(vt3.tolist())
print('\tGBRBM_Second_layer test loss : ', str(test_loss / epoch_cnt))

	GBRBM_First_layer test loss :  tensor(0.0020, grad_fn=<DivBackward0>)
	GBRBM_Second_layer test loss :  tensor(0.0011, grad_fn=<DivBackward0>)
	GBRBM_Second_layer test loss :  tensor(0.0070, grad_fn=<DivBackward0>)


In [13]:
lin = nn.Linear(len(output_from_third), 4)
linear_in = torch.tensor(output_from_third)
print("Linear_in size : {}, Linear_in dim : {}".format(linear_in.size(), linear_in.dim()))

print(lin(linear_in))

Linear_in size : torch.Size([951, 10]), Linear_in dim : 2


RuntimeError: mat1 and mat2 shapes cannot be multiplied (951x10 and 951x4)